<a href="https://colab.research.google.com/github/YasmineSarraj/M3-Assignments-Deep-Learning/blob/main/sBERT_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# sBERT

In [3]:
!pip install sentence_transformers -q

In [4]:
from sentence_transformers import SentenceTransformer, util
from PIL import Image

In [5]:
#Load CLIP model
model = SentenceTransformer('clip-ViT-B-32')


# Request images

In [8]:
import requests
import os
import numpy as np
import heapq

In [47]:
urls = ["https://img01.ztat.net/article/spp-media-p1/98aafc2a3db0358ba3e56f0a953369ad/ce9839c512e844dc8665edf621b260f9.png?imwidth=1800&filter=packshot",
        "https://img01.ztat.net/article/spp-media-p1/b426f4dde3d84a0cb4be86515c06cd23/2f01364a8fe44d73a66023d3213b86b2.jpg?imwidth=1800&filter=packshot",
        "https://img01.ztat.net/article/spp-media-p1/dca555fb7592403eb2efe11fcd61c699/8673e8260d9a41ac80f45829b8de44c8.jpg?imwidth=1800&filter=packshot",
        "https://img01.ztat.net/article/spp-media-p1/c2b6bf0b884c4a2f879efb2374b698b1/0cf250e9be544e67ab71383867516130.jpg?imwidth=1800&filter=packshot",
        "https://img01.ztat.net/article/spp-media-p1/c2b6bf0b884c4a2f879efb2374b698b1/0cf250e9be544e67ab71383867516130.jpg?imwidth=1800&filter=packshot",
        "https://img01.ztat.net/article/spp-media-p1/5fcf853bb4304ef7b6326a33b4178dce/6023c2fc1f2946a6b717c6e94b31177c.jpg?imwidth=1800&filter=packshot",
        "https://img01.ztat.net/article/spp-media-p1/7cbab6736a45460a8cef121d9a1af366/7b995167128f4690add81935a5c6e1eb.jpg?imwidth=1800&filter=packshot"]

In [48]:
dress_urls = ["https://en.zalando.de/levisr-selma-dress-denim-dress-going-steady-le221c01r-k11.html",
              "https://en.zalando.de/buffalo-maxi-dress-blau-bedruckt-bu321c01f-k11.html",
              "https://en.zalando.de/nike-sportswear-midi-dress-shift-dress-whiteblack-ni121c04e-a11.html",
              "https://en.zalando.de/adidas-originals-next-dress-day-dress-black-ad121c0ch-q11.html",
              "https://en.zalando.de/champion-jersey-dress-pink-c7641l002-j11.html",
              "https://en.zalando.de/buffalo-jersey-dress-schwarz-rosa-bedruckt-bu321c00p-q11.html",
              "https://en.zalando.de/adidas-originals-shirt-dress-off-white-ad121c079-a11.html"]

In [49]:
print(len(urls))
print(len(dress_urls))

7
7


In [50]:
for idx, url in enumerate(urls):
  with open(f"/content/imgs/dress_{idx}.jpg", "wb") as f:
    img_data = requests.get(url).content
    f.write(img_data)

# Encode images

In [51]:
imgs_enc = []
img_paths = os.listdir("/content/imgs")

for img_path in img_paths:
  img_emb = model.encode(Image.open(f"/content/imgs/{img_path}"))
  imgs_enc.append(img_emb)

In [52]:
prompt = "summer dress"

In [53]:
cos_sims = util.cos_sim(model.encode(prompt), imgs_enc)[0]
cos_sims

tensor([0.2145, 0.2108, 0.2569, 0.2145, 0.2330, 0.2697, 0.2621])

In [54]:
heapq.nlargest(3, range(len(cos_sims)), cos_sims.__getitem__)

[5, 6, 2]

# Grad.io interface

In [55]:
import gradio as gr

In [56]:
  # best_fits = heapq.nlargest(3, range(len(cos_sims)), cos_sims.__getitem__)
  # best_fit = np.argmax(cos_sims).item()


In [58]:
def show_best_img(prompt):
  cos_sims = util.cos_sim(model.encode(prompt), imgs_enc)[0]
  best_fits = heapq.nlargest(3, range(len(cos_sims)), cos_sims.__getitem__)
  return f"/content/imgs/dress_{best_fits[0]}.jpg", dress_urls[best_fits[0]], f"/content/imgs/dress_{best_fits[1]}.jpg", dress_urls[best_fits[1]], f"/content/imgs/dress_{best_fits[2]}.jpg", dress_urls[best_fits[2]]

demo = gr.Interface(
    fn=show_best_img,
    inputs=gr.Textbox(lines=1, placeholder="Describe the dress you are looking for ..."),
    outputs=["image", "text", "image", "text", "image", "text"]
)
demo.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>